In [ ]:
!pip install timm
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.9 MB/s eta 0:00:00


In [ ]:
descriptions = pd.read_csv('/content/Category Description.csv').iloc[:10]

In [ ]:
descriptions

,Catgory,Prompt 1,Prompt 2,Prompt 3,Prompt 4
0,Airplane,An airplane is a powered flying vehicle with f...,"An airplane is a complex, powered aircraft des...","An airplane is comprised of a fuselage, wings,...",An airplane is an intricate assembly of variou...
1,Automobile,An automobile is a wheeled motor vehicle typic...,"An automobile, commonly known as a car, is a s...",An automobile typically consists of an engine ...,An automobile is a complex assembly of various...
2,Bird,"A bird is a warm-blooded, feathered vertebrate...",A bird is a type of warm-blooded vertebrate ch...,"A bird typically has feathers, wings, a beak, ...",A bird's anatomy is a unique and specialized a...
3,Cat,"A cat is a small, domesticated, carnivorous ma...","A cat, scientifically known as Felis catus, is...","A cat has fur, whiskers, retractable claws, sh...","A cat, a small carnivorous mammal, possesses s..."
4,Deer,"A deer is a hoofed, herbivorous mammal known f...","A deer is a member of the Cervidae family, a g...","A deer typically has a slender body, fur, antl...",A deer is an elegant and adaptable animal with...
5,Dog,A dog is a domesticated mammal known for its l...,"A dog, scientifically known as Canis lupus fam...","A dog typically has fur, a tail, four legs, pa...","A dog, a domesticated canine and a popular pet..."
6,Frog,"A frog is a small, tailless amphibian with lon...","A frog is a member of the amphibian class, par...","A frog has a short, moist body, long hind legs...","A frog, a small and versatile amphibian, posse..."
7,Horse,"A horse is a large, herbivorous mammal known f...","A horse is a large, domesticated mammal known ...","A horse has a large body covered in fur, a lon...","A horse, a large and powerful herbivorous mamm..."
8,Ship,A ship is a large watercraft designed for mari...,"A ship is a substantial watercraft, significan...","A ship typically consists of a hull, deck, pro...","A ship, a complex and large watercraft, compri..."
9,truck,A truck is a motorized vehicle designed primar...,A truck is a motor vehicle designed for the ef...,A truck typically consists of a cab for the dr...,A truck is a versatile motor vehicle with seve...


In [ ]:
l2c = dict(descriptions.Catgory)
l2c

{0: 'Airplane',
 1: 'Automobile',
 2: 'Bird',
 3: 'Cat',
 4: 'Deer',
 5: 'Dog',
 6: 'Frog',
 7: 'Horse',
 8: 'Ship',
 9: 'truck'}

### Offline Text Encoder

**BERT**

In [ ]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
def bert_tokenize(col):
    sentences = col.tolist()
    return tokenizer(sentences, padding=True, return_tensors="pt")

In [ ]:
original_text_tokens = bert_tokenize(descriptions.iloc[:,1])['input_ids']
original_attention_masks = bert_tokenize(descriptions.iloc[:,1])['attention_mask']

In [ ]:
text_tokens = bert_tokenize(descriptions.iloc[:,1])['input_ids']

In [ ]:
attention_masks = bert_tokenize(descriptions.iloc[:,1])['attention_mask']

### DataLoaders

In [ ]:
import numpy as np

import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

from torch.utils.data import Dataset, DataLoader

class Image_text_dataset(Dataset):
    def __init__(self, img_set, text_embeddings, attention_masks):
        self.img_set = img_set
        self.text_tokens = text_tokens
        self.attention_masks = attention_masks

    def __len__(self):
        return len(self.text_tokens)

    def __getitem__(self, idx):
        img, label = self.img_set[idx]
        text_tokens = self.text_tokens[label]
        attention_masks = self.attention_masks[label]
        return img, label, text_tokens, attention_masks


def get_train_valid_loader(data_dir,
                           batch_size,
                           augment=False,
                           random_seed=0,
                           valid_size=0.2,
                           shuffle=True):


    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    valid_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
    ])
    if augment:
        train_transform = transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize
        ])
    else:
        train_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])

    # load the dataset
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=train_transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=train_transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
         Image_text_dataset(train_dataset, text_tokens, attention_masks), batch_size=batch_size, sampler=train_sampler
    )
    valid_loader = torch.utils.data.DataLoader(
        Image_text_dataset(valid_dataset, text_tokens, attention_masks), batch_size=batch_size, sampler=valid_sampler
    )

    return (train_loader, valid_loader)


def get_test_loader(data_dir,
                    batch_size,
                    shuffle=True):

    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

    # define transform
    transform = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])

    dataset = datasets.CIFAR10(
        root=data_dir, train=False,
        download=True, transform=transform,
    )

    data_loader = torch.utils.data.DataLoader(
        Image_text_dataset(dataset, text_tokens), batch_size=batch_size, shuffle=shuffle
    )

    return data_loader

In [ ]:
batch_size = 32

train_loader, val_loader = get_train_valid_loader('./data', batch_size, augment=True)

100%|██████████| 170498071/170498071 [00:10<00:00, 15835663.34it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

rn50 = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 74.4MB/s]


In [ ]:
x = torch.rand((3,224,224))
rn50(x.unsqueeze(0)).shape

torch.Size([1, 1000])

### Image and Text Encoder

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import timm
from transformers import DistilBertModel, DistilBertConfig


class ImageEncoder(nn.Module):
    """
    Encode images to a fixed size vector
    """

    def __init__(
        self, model_name='resnet50', pretrained=True, trainable=True
    ):
        super().__init__()
        self.model = timm.create_model(
            model_name, pretrained, num_classes=0, global_pool="avg"
        )
        for idx, p in enumerate(self.model.parameters()):
            if idx == len(list(self.model.parameters())) - 2:
                p.requires_grad = trainable
            else:
                p.requires_grad = False
            #p.requires_grad = True

    def forward(self, x):
        return self.model(x)


class TextEncoder(nn.Module):
    def __init__(self, model_name="distilbert-base-uncased", pretrained=True, trainable=True):
        super().__init__()
        if pretrained:
            self.model = DistilBertModel.from_pretrained(model_name)
        else:
            self.model = DistilBertModel(config=DistilBertConfig())

        for p in self.model.parameters():
            p.requires_grad = trainable

        # CLS token hidden as the sentence embedding
        self.target_token_idx = 0

    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = output.last_hidden_state
        return last_hidden_state[:, self.target_token_idx, :]


class ProjectionHead(nn.Module):
    def __init__(
        self,
        embedding_dim,
        projection_dim=256,
        dropout=0.1
    ):
        super().__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)

    def forward(self, x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)
        return x

### Text Enhanced Image Classifier

In [ ]:
!pip install info-nce-pytorch
from info_nce import InfoNCE, info_nce

In [ ]:
import random

loss_contrastive = InfoNCE(negative_mode='paired')
loss_clf = nn.CrossEntropyLoss()

class CLIPModel(nn.Module):
    def __init__(
        self,
        temperature=1.0,
        image_embedding=2048,
        text_embedding=768,
        clf_loss_weight=0.8
    ):
        super().__init__()
        self.image_encoder = ImageEncoder()
        self.text_encoder = TextEncoder()
        self.image_projection = ProjectionHead(embedding_dim=image_embedding)
        self.text_projection = ProjectionHead(embedding_dim=text_embedding)
        self.temperature = temperature
        self.clf = nn.Linear(image_embedding,10)
        self.clf_loss_weight = clf_loss_weight

    def forward(self, img, label, text, att, evaluation=False):
        #img and text features
        image_features = self.image_encoder(img)
        text_features = self.text_encoder(
            input_ids=text, attention_mask=att
        )
        #project to same dim
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)

        if evaluation: return F.softmax(self.clf(image_features),dim=-1)

        #sample negative texts
        neg_labels = torch.LongTensor([random.choice(list(range(i))+list(range(i+1,10))) for i in labels])
        neg_texts_tokens = original_text_tokens[neg_labels].to(device)
        neg_texts_embeddings = self.text_encoder(neg_texts_tokens, original_attention_masks[neg_labels].to(device))
        neg_texts_embeddings = self.text_projection(neg_texts_embeddings).unsqueeze(1)

        #contrastive loss: InfoNCE
        contrastive_loss = loss_contrastive(image_embeddings, text_embeddings, neg_texts_embeddings)

        #classification loss: CE
        logits = F.softmax(self.clf(image_features),dim=-1)
        clf_loss = loss_clf(logits, labels)

        #combine loss
        loss = (1-self.clf_loss_weight) * contrastive_loss + self.clf_loss_weight * clf_loss

        return logits, loss

In [ ]:
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import classification_report
device = 'cuda' if torch.cuda.is_available() else 'cpu'

EPOCH = 10
model = CLIPModel(clf_loss_weight=0.2).to(device)
optimizer = optim.AdamW(model.parameters(), lr=5e-3)

accs = []
for epoch in range(EPOCH):
  total_loss = 0
  for i, batch in enumerate(tqdm(train_loader)):
      imgs, labels, text_tokens, att = batch

      images = imgs.to(device)
      texts = text_tokens.to(device)
      labels = labels.to(device)
      att = att.to(device)

      optimizer.zero_grad()
      logits, loss = model(images, labels, texts, att)
      loss.backward()
      optimizer.step()

      total_loss+=loss.item()

      if not i%100:
        #validation
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch in val_loader:
                imgs, labels, text_tokens, att = batch

                images = imgs.to(device)
                texts = text_tokens.to(device)
                labels = labels.to(device)
                att = att.to(device)

                logits = model(images, labels, texts, att, evaluation=True)
                pred = logits.argmax(-1)
                correct += (pred==labels).float().sum()

        acc = correct/(32*len(val_loader))
        accs.append(acc)
        print(f'Epoch {epoch+1} Batch {i+1} Training Loss: {total_loss/(i+1)} Validation Accuracy: {acc}')

  print(f'Epoch {epoch+1} Avg Training Loss: {total_loss/len(train_loader)}')

  0%|          | 1/1250 [00:39<13:37:38, 39.28s/it]

Epoch 1 Batch 1 Training Loss: 1.0213509798049927 Validation Accuracy: 0.1843051016330719


  8%|▊         | 101/1250 [01:59<4:08:56, 13.00s/it]

Epoch 1 Batch 101 Training Loss: 0.9187669759929771 Validation Accuracy: 0.8442491888999939


 16%|█▌        | 201/1250 [03:19<3:44:50, 12.86s/it]

...

Epoch 6 Batch 1001 Training Loss: 0.8612604714654661 Validation Accuracy: 0.8990615010261536


 88%|████████▊ | 1101/1250 [15:24<31:50, 12.82s/it]

Epoch 6 Batch 1101 Training Loss: 0.8612682816118246 Validation Accuracy: 0.8999600410461426


 92%|█████████▏| 1151/1250 [15:43<01:21,  1.22it/s]


KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/cv_model.pth')